# 0.0 IMPORTS

In [51]:
import pandas as pd
import matplotlib
import numpy as np
import psycopg2 as pg
import seaborn as sns

from IPython.core.display      import HTML

In [30]:
!pip install seaborn

In [25]:
## Helper Functions

In [52]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


# 1.0 COLLECT DATA AND LOADING DATASET

In [14]:
#credentials
user     = 'member'
pwd      = 'cdspa'
host     = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port     = 5432
database = 'comunidadedsdb'

In [128]:
conn = pg.connect(user=user,
                  password = pwd,
                  host = host,
                  port = port,
                  database = database)

## 1.1 SCHEMA QUERY

In [79]:
cursor = conn.cursor()

query = """ 
    SELECT nspname
    FROM pg_catalog.pg_namespace
"""

df = pd.read_sql_query(query, conn)
cursor.close()
conn.close()
df

,nspname
0,pg_toast
1,pg_temp_1
2,pg_toast_temp_1
3,pg_catalog
4,information_schema
5,public
6,pa004
7,pg_temp_4
8,pg_toast_temp_4
9,pa005


## 1.2 TABLE QUERY


In [47]:
cursor = conn.cursor()

query = """ 
    SELECT tablename
    FROM pg_tables
    WHERE schemaname = 'pa004'
"""

df = pd.read_sql_query(query, conn)
cursor.close()
conn.close()

In [48]:
df

,tablename
0,users
1,vehicle
2,insurance


## 1.3 JOIN TABLES AND CREATE CSV DATAFRAME

In [130]:
cursor = conn.cursor()

query = """ 
    SELECT *
    FROM pa004.users u LEFT JOIN pa004.vehicle v ON (u.id = v.id)
                       LEFT JOIN pa004.insurance i ON (u.id = i.id ) 
    
"""

df = pd.read_sql_query(query, conn)
cursor.close()
conn.close()



df = df.drop(df.columns[[5,9]], axis = 1)
df.to_csv('train',index = False)
# Neste passo, optei por criar um arquivo csv com os dados, mesmo sabendo que nas empresas não se faz isso. 
# Fiz esta opção, pois os dados foram tirados de um banco de dados na AWS, que pertence a comunidade a qual faço parte
# (comunidade ds), e criando este arquivo posso esconder as credenciais de acesso ao bd.

,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Male,44,28.0,26.0,1,> 2 Years,Yes,0,40454.0,217,1
1,Male,76,3.0,26.0,1,1-2 Year,No,0,33536.0,183,0
2,Male,47,28.0,26.0,1,> 2 Years,Yes,0,38294.0,27,1
3,Male,21,11.0,152.0,1,< 1 Year,No,1,28619.0,203,0
4,Female,29,41.0,152.0,1,< 1 Year,No,1,27496.0,39,0


In [135]:
df_raw = pd.read_csv('C:/Users/lucas/OneDrive/Repos/Health-Insurance-Cross-Sell/data/train.csv')

# 2.0 Data Descriptive

In [136]:
df1 = df_raw.copy()

## 2.1 Rename Columns

In [137]:
df1.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

df1.columns



Index(['id', 'gender', 'age', 'region_code', 'policy_sales_channel',
       'driving_license', 'vehicle_age', 'vehicle_damage',
       'previously_insured', 'annual_premium', 'vintage', 'response'],
      dtype='object')

## 2.1 Data Dimensions

In [138]:
print('Number of Rows : {}'.format(df1.shape[0]))
print('Number of columns: {}'.format(df1.shape[1]))

Number of Rows : 381109
Number of columns: 12


## 2.2 Data Types

In [139]:
df1.dtypes

id                       int64
gender                  object
age                      int64
region_code              int64
policy_sales_channel     int64
driving_license          int64
vehicle_age             object
vehicle_damage          object
previously_insured       int64
annual_premium           int64
vintage                  int64
response                 int64
dtype: object

## 2.3 Check NA's

In [140]:
df1.isna().sum()

id                      0
gender                  0
age                     0
region_code             0
policy_sales_channel    0
driving_license         0
vehicle_age             0
vehicle_damage          0
previously_insured      0
annual_premium          0
vintage                 0
response                0
dtype: int64

## 2.4 Data Descriptive

In [141]:
cat_attributes = df1.select_dtypes(include= 'object')
num_attributes = df1.select_dtypes(exclude= 'object')

In [152]:
# Central Tendency
c1 = pd.DataFrame(num_attributes.mean()).T
c2 = pd.DataFrame(num_attributes.median()).T

# Dispersion Measure

d1 = pd.DataFrame(num_attributes.std()).T
d2 = pd.DataFrame(num_attributes.min()).T
d3 = pd.DataFrame(num_attributes.max()).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# Concat and Create dataframe Data Descriptive

m = pd.concat([d2, d3, d4, c1, c2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m


,attributes,min,max,range,mean,median,std,skew,kurtosis
0,id,0.0,381108.0,381108.0,190554.000000,190554.0,110016.836208,9.443274e-16,-1.200000
1,age,20.0,85.0,65.0,38.822584,36.0,15.511611,6.725390e-01,-0.565655
2,region_code,0.0,52.0,52.0,26.388807,28.0,13.229888,-1.152664e-01,-0.867857
3,policy_sales_channel,1.0,163.0,162.0,112.034295,133.0,54.203995,-9.000081e-01,-0.970810
4,driving_license,0.0,1.0,1.0,0.997869,1.0,0.046110,-2.159518e+01,464.354302
5,previously_insured,0.0,1.0,1.0,0.458210,0.0,0.498251,1.677471e-01,-1.971871
6,annual_premium,2630.0,540165.0,537535.0,30564.389581,31669.0,17213.155057,1.766087e+00,34.004569
7,vintage,10.0,299.0,289.0,154.347397,154.0,83.671304,3.029517e-03,-1.200688
8,response,0.0,1.0,1.0,0.122563,0.0,0.327936,2.301906e+00,3.298788
